![alt text](pageheader_rose2_babies.jpg)

# Notebook to produce publication quality graphs for the alarm review paper

Author: Gusztav Belteki MD, PhD, FRCPCH

Contact: gbelteki@aol.com

This Notebook contains the code to generate the images of this paper: Mitra N, Belteki G. __Fifteen-minute consultation: How to interpret and manage ventilator alarms in the neonatal intensive care unit.__ _Arch Dis Child Educ Pract Ed._ 2020 Oct 13:edpract-2019-318242. doi: 10.1136/archdischild-2019-318242. Epub ahead of print. PMID: 33051186.

The paper is available here (full text requires access to the journal): [link](https://ep.bmj.com/content/early/2020/10/13/archdischild-2019-318242)

The outputs (numbers, tables, graphs) of this Notebook have been suppressed to comply with copyrights. The corresponding data and graphs can be found in the paper.

#### Author: Dr Gusztav Belteki

### Importing the required libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import sys
import pickle
import scipy as sp
from scipy import stats
from pandas import Series, DataFrame
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import dates
from datetime import datetime, timedelta
from collections import defaultdict

%matplotlib inline

matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)

In [ ]:
print("Python version: {}".format(sys.version))
print("IPython version: {}".format(IPython.__version__))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))

### Import modules containing own functions

In [ ]:
from gb_loader import *
from gb_stats import *
from gb_transform import *

### List and set the working directory and the directories to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'alarms_review'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_draeger'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Draeger/service_evaluation_old' % DRIVE

# Directory to write results and selected images to 
if not os.path.isdir('%s/%s/%s' % (CWD, 'Analyses', TOPIC)):
    os.makedirs('%s/%s/%s' % (CWD, 'Analyses', TOPIC))
DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', TOPIC)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ, DIR_WRITE, DATA_DUMP

### List of the  recordings

In [ ]:
# One recording from each patient, all of them 24 hours old or longer
# The sub folders containing the individual recordings have the same names within cwd

recordings = ['DG030', 'DG035', 'DG044']

### Import clinical details

In [ ]:
# Clinical details of the both service evaluation
clinical_details = pd.read_excel('/Users/guszti/ventilation_draeger/service_evaluation_patient_data.xlsx')
clinical_details.index = clinical_details['Recording']

# Limit clinical details to the second service evaluation
clinical_details = clinical_details.loc[recordings]

clinical_details.info()

In [ ]:
current_weights = {}
for recording in recordings:
    current_weights[recording] = clinical_details.loc[recording, 'Current weight' ] / 1000

### Import ventilator parameters retrieved with 1/sec frequency

In [ ]:
slow_measurements = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = slow_measurement_finder(flist)
    print('Loading recording %s' % recording)
    print(files)
    fnames = ['%s/%s/%s' % (DIR_READ, recording, filename) for filename in files]
    slow_measurements[recording] =  data_loader(fnames)

### Calculating parameters / body weight kg

In [ ]:
for recording in recordings:
    try:
        a = slow_measurements[recording]
        a['VT_kg']       = a['5001|VT [mL]'] / current_weights[recording]
        a['VTi_kg']      = a['5001|VTi [mL]'] / current_weights[recording]
        a['VTe_kg']      = a['5001|VTe [mL]'] / current_weights[recording]
        a['VTmand_kg']   = a['5001|VTmand [mL]'] / current_weights[recording]
        a['VTspon_kg']   = a['5001|VTspon [mL]'] / current_weights[recording]
        a['VTimand_kg']  = a['5001|VTimand [mL]'] / current_weights[recording]
        a['VTemand_kg']  = a['5001|VTemand [mL]'] / current_weights[recording]
        a['VTispon_kg']  = a['5001|VTispon [mL]'] / current_weights[recording]
        a['VTespon_kg']  = a['5001|VTespon [mL]'] / current_weights[recording]
    except KeyError:
        # print('%s does not have all of the parameters' % recording)
        pass

In [ ]:
for recording in recordings:
    try:
        a = slow_measurements[recording]
        a['VThf_kg']      = a['5001|VThf [mL]'] / current_weights[recording]
        a['DCO2_corr_kg'] = a['5001|DCO2 [10*mL^2/s]'] * 10 / (current_weights[recording]) ** 2
    except KeyError:
        # print('%s does not have all of the parameters' % recording)
        pass

In [ ]:
for recording in recordings:
    try:
        a = slow_measurements[recording]
        a['MV_kg'] =      a['5001|MV [L/min]'] / current_weights[recording]
        a['MVi_kg'] =     a['5001|MVi [L/min]'] / current_weights[recording]
        a['MVe_kg'] =     a['5001|MVe [L/min]'] / current_weights[recording]
        a['MVemand_kg'] = a['5001|MVemand [L/min]'] / current_weights[recording]
        a['MVespon_kg'] = a['5001|MVespon [L/min]'] / current_weights[recording]
        a['MVleak_kg'] =  a['5001|MVleak [L/min]'] / current_weights[recording]
    except KeyError:
        # print('%s does not have all of the parameters' % recording)
        pass

### Resampling to remove half-empty rows

In [ ]:
%%time

# 1/sec data are retrieved in two parts which need to be joined
# This resampling steps combines the two parts

for recording in recordings:
    slow_measurements[recording] = slow_measurements[recording].resample('1S').mean()

### Calculate recording durations

In [ ]:
# Time stamps are obtained from 'slow measurements' 

recording_duration = {}
for recording in recordings:
    recording_duration[recording] = slow_measurements[recording].index[-1] - slow_measurements[recording].index[0]

In [ ]:
recording_duration_seconds = {}
recording_duration_hours = {}

for recording in recordings:
    temp = recording_duration[recording]
    recording_duration_seconds[recording] = temp.total_seconds()
    recording_duration_hours[recording] = temp.total_seconds() / 3600

### Import ventilator modes and settings

##### Import ventilation settings

In [ ]:
vent_settings = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = slow_setting_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ, recording, filename) for filename in files]
    vent_settings[recording] =  data_loader(fnames)

In [ ]:
# remove less important ventilator settings to simplify the table

vent_settings_selected = {}
for recording in recordings:
    vent_settings_selected[recording] = vent_settings_cleaner(vent_settings[recording])

In [ ]:
# Create a another dictionary of Dataframes wit some of the ventilation settings (set VT, set RR, set Pmax)

lsts = [(['VT_weight'], ['VTi', 'VThf']), (['RR_set'], ['RR']), (['Pmax'], ['Pmax', 'Ampl hf max'])]
vent_settings_2 = {}
                                                            
for recording in recordings:
    frmes = []                                                       
    for name, pars in lsts:
        if pars in [['VTi', 'VThf']]:
            ind = []
            val = []
            for index, row in vent_settings_selected[recording].iterrows():
                if row['Id'] in pars:
                    ind.append(index)
                    val.append(row['Value New'] / current_weights[recording])
                frmes.append(DataFrame(val, index = ind, columns = name))
        else:
            ind = []
            val = []
            for index, row in vent_settings_selected[recording].iterrows():
                if row['Id'] in pars:
                    ind.append(index)
                    val.append(row['Value New'])
                frmes.append(DataFrame(val, index = ind, columns = name))
    vent_settings_2[recording] = pd.concat(frmes, sort = False)
    vent_settings_2[recording].drop_duplicates(inplace = True)

##### Import ventilation modes

In [ ]:
vent_modes = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = slow_text_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ, recording, filename) for filename in files]
    vent_modes[recording] =  data_loader(fnames)

In [ ]:
# remove less important ventilator mode settings to simplify the table

vent_modes_selected = {}
for recording in recordings:
    vent_modes_selected[recording] = vent_mode_cleaner(vent_modes[recording])

### Import alarm settings

In [ ]:
alarm_settings = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = alarm_setting_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ, recording, filename) for filename in files]
    alarm_settings[recording] =  data_loader(fnames)

In [ ]:
# Remove etCO2 limits which were not used

alarm_settings_selected = {}

for recording in recordings:
    alarm_settings_selected[recording] = alarm_setting_cleaner(alarm_settings[recording])

In [ ]:
# Create a another dictionary of Dataframes with some of the alarm settings 

lsts = [(['MV_high_weight'], ['MVe_HL']), (['MV_low_weight'], ['MVe_LL']), 
        (['PIP_high'], ['PIP_HL']), (['RR_high'], ['RR_HL'])]
alarm_settings_2 = {}
                                                            
for recording in recordings:
    frmes = []                                                       
    for name, pars in lsts:
        if pars in [['MVe_HL'], ['MVe_LL']]:
            ind = []
            val = []
            for index, row in alarm_settings_selected[recording].iterrows():
                if row['Id'] in pars:
                    ind.append(index)
                    val.append(row['Value New'] / current_weights[recording])
                frmes.append(DataFrame(val, index = ind, columns = name))
        else:
            ind = []
            val = []
            for index, row in alarm_settings_selected[recording].iterrows():
                if row['Id'] in pars:
                    ind.append(index)
                    val.append(row['Value New'])
                frmes.append(DataFrame(val, index = ind, columns = name))
    alarm_settings_2[recording] = pd.concat(frmes, sort = False)
    alarm_settings_2[recording].drop_duplicates(inplace = True)

### Import alarm states

In [ ]:
alarm_states = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = alarm_state_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ, recording, filename) for filename in files]
    alarm_states[recording] =  data_loader(fnames)

### Generate alarm events from alarm states

In [ ]:
# Define function to retrieve alarm events from alarm timing data
def alarm_events_calculator(dframe, al):
    
    '''
    DataFrame, str -> DataFrame
    
    dframe: DataFrame containing alarm states
    al:  alarm category (string)
    
    Returns a pd.DataFrame object with the time stamps when the alarm went off and the duration (in seconds)
    of the alarm for alarm 'al' in recording 'rec'
    '''
    
    alarms = dframe
    alarm = alarms[alarms.Name == al]
    length = len(alarm)
    delta = np.array([(alarm.Date_Time[i] - alarm.Date_Time[i-1]).total_seconds() 
            for i in range(1, length) if alarm['State New'][i] == 'NotActive' and alarm['State New'][i-1] == 'Active'])
    stamp = np.array([alarm.index[i-1] 
            for i in range(1, length) if alarm['State New'][i] == 'NotActive' and alarm['State New'][i-1] == 'Active'])
    data = {'duration_seconds': delta, 'time_went_off': stamp,}
    alarm_t = DataFrame(data, columns = ['time_went_off', 'duration_seconds'])
    return alarm_t

Using the files containing the alarm states, for each **alarm category** in **each recording** create a DataFrame with the **timestamps** the alarm went off and the **duration** of the alarm and store them in a dictionary of dictionaries 

In [ ]:
# Create a list of alarms occurring during each recording

alarm_list = {}

for recording in recordings:
    alarm_list[recording] = sorted(set(alarm_states[recording].Name))

In [ ]:
alarm_events = {}

for recording in recordings:
    alarm_events[recording] = {}
    for alarm in alarm_list[recording]:
        alarm_events[recording][alarm] = alarm_events_calculator(alarm_states[recording], alarm) 

### Calculate descriptive statistics for each alarm in each recording and write them to file

In [ ]:
def alarm_stats_calculator(dframe, rec, al):
    '''
    dframe: DataFrame containing alarm events 
    rec: recording (string)
    al: alarm (string)
    
    Returns detailed statistics about a particular alarm (al) in a particular recording (rec);
    - number of times alarm went off and its value normalized to 24 hour periods
    - mean, median, standard deviation, mean absolute deviation, minimum, 25th centile, 75th centile, maximum
    time period when the alarm was off
    - the total amount of time the alarm was off and its relative value in percent as the total recording time 
    '''
    alarm = dframe[al].duration_seconds
    return  (alarm.size, round((alarm.size / (recording_duration_hours[rec] / 24)), 1), 
            round(alarm.mean() , 1), round(alarm.median(), 1), round(alarm.std(), 1), round(alarm.min() , 1), 
            round(alarm.quantile(0.25), 1), round(alarm.quantile(0.75), 1), round(alarm.max(), 1),
            round(alarm.sum(), 1), round(alarm.sum() * 100 / recording_duration_seconds[rec] ,3))

In [ ]:
alarm_stats = {}

for recording in recordings:
    alarm_stats[recording] = {}
    for alarm in alarm_list[recording]:
        data = alarm_stats_calculator(alarm_events[recording], recording, alarm)
        frame = DataFrame([data], columns = ['number of events', 'number of event per 24h', 
                                             'mean duration (s)', 'median duration (s)', 'SD duration (s)', 
                                             'miminum duration (s)', 
                                             'duration 25th centile (s)', 'duration 75th centile (s)', 
                                             'maximum duration (s)', 'cumulative duration (s)', 
                                             'percentage of recording length (%)'], index = [alarm])
        alarm_stats[recording][alarm] = frame

In [ ]:
stats_all = {}

for recording in recordings:
    stats = []
    for alarm in alarm_stats[recording]:
        stats.append(alarm_stats[recording][alarm])
    stats_all[recording] = pd.concat(stats)  

In [ ]:
# Write descriptive statistics in a multisheet Excel file

writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'alarm_stats.xlsx'))


for recording in recordings:
    stats_all[recording].to_excel(writer, recording)
writer.save()

### Visualise alarm statistics for the individual alarms in the individual recording

In [ ]:
# Generates a plot with the cumulative times (in seconds) of the various alarm occurring during recording (rec).
# Displays the plot

def alarm_plot_1(rec, write = True, show = True, dpi = 300, filetype = 'jpg'):
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
    ax1 = fig.add_subplot(1, 1, 1)
    xs = [i + 0.1 for i, _ in enumerate(alarm_list[rec])]
    stats = []
    for alarm in alarm_list[rec]:
        stats.append(alarm_stats[rec][alarm]['cumulative duration (s)'])
    stats_all = pd.concat(stats)
    plt.barh(xs, stats_all, color = 'red')
    plt.xlabel("seconds", fontsize  = 12)
    #plt.title("Recording %s : How long was the alarm active over the %d seconds of recording?" % (rec, 
                #recording_duration_seconds[rec]), fontsize = 12)
    plt.yticks([i + 0.5 for i, _ in enumerate(alarm_list[rec])], alarm_list[rec], fontsize = 12)
    plt.xticks(fontsize = 12)
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, 'alarm_durations_1', rec), dpi=dpi, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=filetype,
        transparent=False, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
alarm_plot_1('DG030')

In [ ]:
alarm_plot_1('DG035')

In [ ]:
alarm_plot_1('DG044')

In [ ]:
# Generates a plot with the cumulative times (expressed as percentage of the total recording time) 
# of the various alarm occurring during recording (rec).
# Displays the plot

def alarm_plot_2(rec, write = True, show = True, dpi = 300, filetype = 'jpg'):
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
    ax1 = fig.add_subplot(1, 1, 1)
    xs = [i + 0.1 for i, _ in enumerate(alarm_list[rec])]
    stats = []
    for alarm in alarm_list[rec]:
        stats.append(alarm_stats[rec][alarm]['percentage of recording length (%)'])
    stats_all = pd.concat(stats)
    plt.barh(xs, stats_all, color = 'red')
    plt.xlabel("% of total recording time", fontsize  = 12)
    #plt.title("Recording %s: How long the alarm active over the %s hours of recording?" % (rec, 
      #      str(recording_duration[rec])), fontsize = 12)
    plt.yticks([i + 0.5 for i, _ in enumerate(alarm_list[rec])], alarm_list[rec], fontsize = 12)
    plt.xticks(fontsize = 12)
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, 'alarm_durations_2', rec), dpi=dpi, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=filetype,
        transparent=False, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

In [ ]:
alarm_plot_2('DG030')

In [ ]:
alarm_plot_2('DG035')

In [ ]:
alarm_plot_2('DG044')

In [ ]:
# Displays the individual alarm events of the recording (rec) along the time axis
# Displays the plot

def alarm_plot_3(rec, write = True, show = True, dpi = 300, filetype = 'jpg'):
    alarm_state = alarm_states[rec]
    numbered = Series(np.zeros(len(alarm_state)), index = alarm_state.index)
    for i in range(1, len(alarm_state)):
        if alarm_state.iloc[i]['State New'] == 'Active':
            numbered[i] = alarm_list[rec].index(alarm_state.iloc[i]['Id']) + 1
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
    ax1 = fig.add_subplot(1, 1, 1);

    ax1.plot(alarm_state.index,  numbered, '|', color = 'red', markersize = 16, markeredgewidth = 1,)
    plt.xlabel("Time", fontsize  = 12)
   # plt.title("Alarm events during recording %s" % rec , fontsize = 12)
    plt.yticks([i+1  for i, _ in enumerate(alarm_list[rec])], alarm_list[rec], fontsize = 12);
    plt.xticks(fontsize = 12, rotation = 0)
    plt.ylim(0.5, len(alarm_list[rec]) + 0.5)
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax1.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, 'alarm_durations_3', rec), dpi=dpi, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=filetype,
        transparent=False, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close()

##### Example plots

In [ ]:
alarm_plot_3('DG030')

In [ ]:
alarm_plot_3('DG035')

In [ ]:
alarm_plot_3('DG044')

### Investigate the relationship of MV and RR parameter readings, ventilation settings and alarm settings

In [ ]:
for recording in recordings:
    slow_measurements[recording] = pd.concat([slow_measurements[recording],
            vent_settings_2[recording], alarm_settings_2[recording]], axis = 0, join = 'outer', sort = False)
    slow_measurements[recording].sort_index(inplace = True)

In [ ]:
for recording in recordings:
    slow_measurements[recording] = slow_measurements[recording].fillna(method = 'pad')

In [ ]:
def minute_volume_plotter(rec, ylim = False, write = True, show = True, dpi = 300, filetype = 'jpg'):
    '''
    Plots the total minute volumme (using the data obtained with 1/sec sampling rate) 
    together with the "MV low" and "MV high" alarm limits
    Displays the plot
    
    '''
    if ylim:
        ymax =  ylim
    else:
        ymax = slow_measurements[rec]['MV_high_weight'].max() + 0.3
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
    ax1 = fig.add_subplot(1, 1, 1);
    
    slow_measurements[rec]['MV_kg'].plot(ax = ax1, color = 'blue', ylim = [0, ymax], x_compat = True );
    slow_measurements[rec]['MV_low_weight'].plot(ax = ax1, color = 'green', linewidth = 3, ylim = [0, ymax], 
                                                 x_compat = True);
    slow_measurements[rec]['MV_high_weight'].plot(ax = ax1, color = 'red', linewidth = 3, ylim = [0, ymax], 
                                                  x_compat = True );
    
    #ax1.set_title('Minute volume - %s' % rec, size = 12, color = 'black')
    ax1.set_xlabel('Time', size = 12, color = 'black')
    ax1.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax1.legend(['MV', 'low alarm limit', 'high alarm limit'], fontsize = 12, ncol  = 3)
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax1.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, rec, 'minute_volume_with_alarm_limit'), dpi=dpi, 
        format=filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close();

In [ ]:
minute_volume_plotter('DG030')

In [ ]:
minute_volume_plotter('DG035')

In [ ]:
minute_volume_plotter('DG044')

In [ ]:
def resp_rate_plotter(rec, ylim = False, write = True, show = True, dpi = 300, filetype = 'jpg'):
    '''
    Plots the total reapiratory rate (using the data obtained with 1/sec sampling rate) 
    together with the set backup rate and "RR high" alarm limits
    Displays the plot
    
    '''
    if ylim:
        ymax =  ylim
    else:
        ymax = slow_measurements[rec]['5001|RR [1/min]'].max() + 10
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
    wspace=None, hspace=0.7)
    ax1 = fig.add_subplot(1, 1, 1);
    
    slow_measurements[rec]['5001|RR [1/min]'].plot(ax = ax1, color = 'blue', ylim = [0, ymax], x_compat = True);
    slow_measurements[rec]['RR_high'].plot(ax = ax1, color = 'red', linewidth = 3, ylim = [0, ymax], x_compat = True );
    slow_measurements[rec]['RR_set'].plot(ax = ax1, color = 'green', linewidth = 3, ylim = [0, ymax], x_compat = True );
    
    #ax1.set_title('Respiratory rate - %s' % rec, size = 12, color = 'black')
    ax1.set_xlabel('Time', size = 12, color = 'black')
    ax1.set_ylabel('breaths/min', size = 12, color = 'black')
    ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    ax1.legend(['RR', 'high alarm limit', 'set RR'], fontsize = 12, ncol = 3)
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax1.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, rec, 'resp_rate_with_alarm_limit'), dpi=dpi, 
        format=filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close();

In [ ]:
resp_rate_plotter('DG030', ylim = 180)

In [ ]:
resp_rate_plotter('DG035', ylim = 180)

In [ ]:
resp_rate_plotter('DG044', ylim = 180)

In [ ]:
def pressure_plotter(rec, ylim = False, write = True, show = True, dpi = 300, filetype = 'jpg'):
    
    if ylim:
        ymax =  ylim
    else:
        ymax = slow_measurements[rec]['5001|PIP [mbar]'].max() + 10
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements[rec]['5001|PIP [mbar]'].plot(ax = ax, color = 'red', ylim = [0, ymax], x_compat = True,)
    slow_measurements[rec]['Pmax'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True )
    
    # If you want to highlight sub-periods on the graphs, uncomment these lines and 
    # insert date and time in the correct format
    
    #ax.axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
    #ax.axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'Pmax', ], ncol = 2, fontsize = 12,)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, rec, 'PIP_Pmax'), dpi=dpi, 
        format=filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close();

In [ ]:
pressure_plotter('DG030')

In [ ]:
pressure_plotter('DG035', ylim = 50)

In [ ]:
pressure_plotter('DG044')

In [ ]:
def volume_plotter(rec, ylim = False, write = True, show = True, dpi = 300, filetype = 'jpg'):

    if ylim:
        ymax =  ylim
    else:
        ymax = slow_measurements[rec]['VTmand_kg'].max() + 10
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements[rec]['VTmand_kg'].plot(ax = ax, color = 'blue', ylim = [0, ymax], x_compat = True,)
    slow_measurements[rec]['VT_weight'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True )
    
    # If you want to highlight sub-periods on the graphs, uncomment these lines and 
    # insert date and time in the correct format
    
    #ax.axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
    #ax.axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
    #ax.text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTe', 'VT target', ], ncol = 2, fontsize = 12,)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if write:
        fig.savefig('%s/%s_%s.jpg' % (DIR_WRITE, rec, 'VT_VTset'), dpi=dpi, 
        format=filetype, bbox_inches='tight', pad_inches=0.1)
        
    if not show:
        plt.close();

In [ ]:
volume_plotter('DG030', ylim = 20)

In [ ]:
volume_plotter('DG035', ylim = 20)

In [ ]:
volume_plotter('DG044', ylim = 25)

## Create the tables and figures of the paper

### Figure 1

#### Figure 1A

In [ ]:
dpi = 300
filetype = 'jpg'

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)

slow_measurements['DG035']['2016-04-19 06:00:00':]['5001|PIP [mbar]'].plot(ax = ax, 
                                                            color = 'red', x_compat = True,)
slow_measurements['DG035']['2016-04-19 06:00:00':]['Pmax'].plot(ax = ax, 
                                                           color = 'black', linewidth = 2, x_compat = True )
ax.set_ylim(0, 50)
ax.set_xlabel('Time', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('',  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax', ], ncol = 2, fontsize = 12,)
#ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

majorFmt = dates.DateFormatter('%H:%M')  
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1A', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 1B

In [ ]:
dpi = 300
filetype = 'jpg'

alarm_state = alarm_states['DG035']
numbered = Series(np.zeros(len(alarm_state)), index = alarm_state.index)
for i in range(1, len(alarm_state)):
    if alarm_state.iloc[i]['State New'] == 'Active':
        numbered[i] = alarm_list['DG035'].index(alarm_state.iloc[i]['Id']) + 1

numbered = numbered['2016-04-19 06:00:00':]

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);

ax1.plot(numbered.index,  numbered, '|', color = 'red', markersize = 16, markeredgewidth = 1,)
plt.xlabel("Time", fontsize  = 12)
plt.yticks([i+1  for i, _ in enumerate(alarm_list['DG035'])], alarm_list['DG035'], fontsize = 12);
plt.xticks(fontsize = 12, rotation = 0)
plt.ylim(0.5, len(alarm_list['DG035']) + 0.5)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1B', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 1 color

In [ ]:
dpi = 200
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 9)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=0.2)

# Figure 1A

ax = fig.add_subplot(2, 1, 1)

slow_measurements['DG035']['2016-04-19 06:00:00':]['5001|PIP [mbar]'].plot(ax = ax, 
                                                    color = 'red', x_compat = True,)
slow_measurements['DG035']['2016-04-19 06:00:00':]['Pmax'].plot(ax = ax, 
                                                    color = 'black', linewidth = 2, x_compat = True )
ax.set_ylim(0, 50)
ax.set_xlabel('', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('',  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax', ], ncol = 2, fontsize = 12,)
#ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

majorFmt = dates.DateFormatter('%H:%M')  
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    

# Figure 1B

alarm_state = alarm_states['DG035']
numbered = Series(np.zeros(len(alarm_state)), index = alarm_state.index)
for i in range(1, len(alarm_state)):
    if alarm_state.iloc[i]['State New'] == 'Active':
        numbered[i] = alarm_list['DG035'].index(alarm_state.iloc[i]['Id']) + 1
    
numbered = numbered['2016-04-19 06:00:00':]
    
ax1 = fig.add_subplot(2, 1, 2);

ax1.plot(numbered.index,  numbered, '|', color = 'red', markersize = 16, markeredgewidth = 1,)
plt.xlabel("Time", fontsize  = 12)
plt.yticks([i+1  for i, _ in enumerate(alarm_list['DG035'])], alarm_list['DG035'], fontsize = 12);
plt.xticks(fontsize = 12, rotation = 0)
plt.ylim(0.5, len(alarm_list['DG035']) + 0.5)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.0, 0.92, 'A', fontsize = 20) 
fig.text(0.0, 0.54, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1_col', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 1 bw

In [ ]:
dpi = 200
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 9)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=0.2)

# Figure 1A

ax = fig.add_subplot(2, 1, 1)

slow_measurements['DG035']['2016-04-19 06:00:00':]['5001|PIP [mbar]'].plot(ax = ax, 
                                        color = 'black', alpha = 0.5, x_compat = True,)
slow_measurements['DG035']['2016-04-19 06:00:00':]['Pmax'].plot(ax = ax, 
                                        color = 'black', linewidth = 2, x_compat = True )
ax.set_ylim(0, 50)
ax.set_xlabel('', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('',  size = 12, color = 'black')
ax.legend(['PIP', 'Pmax', ], ncol = 2, fontsize = 12,)
#ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

majorFmt = dates.DateFormatter('%H:%M')  
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    

# Figure 1B

alarm_state = alarm_states['DG035']
numbered = Series(np.zeros(len(alarm_state)), index = alarm_state.index)
for i in range(1, len(alarm_state)):
    if alarm_state.iloc[i]['State New'] == 'Active':
        numbered[i] = alarm_list['DG035'].index(alarm_state.iloc[i]['Id']) + 1
    
numbered = numbered['2016-04-19 06:00:00':]
    
ax1 = fig.add_subplot(2, 1, 2);

ax1.plot(numbered.index,  numbered, '|', color = 'black', markersize = 16, markeredgewidth = 1,)
plt.xlabel("Time", fontsize  = 12)
plt.yticks([i+1  for i, _ in enumerate(alarm_list['DG035'])], alarm_list['DG035'], fontsize = 12);
plt.xticks(fontsize = 12, rotation = 0)
plt.ylim(0.5, len(alarm_list['DG035']) + 0.5)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.0, 0.92, 'A', fontsize = 20) 
fig.text(0.0, 0.54, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1_bw', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 2A

In [ ]:
dpi = 300
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
    
slow_measurements['DG044']['MV_kg'].plot(ax = ax1, color = 'blue', x_compat = True )
slow_measurements['DG044']['MV_low_weight'].plot(ax = ax1, color = 'green', linewidth = 3, x_compat = True)
slow_measurements['DG044']['MV_high_weight'].plot(ax = ax1, color = 'red', linewidth = 3,  x_compat = True)

ax1.set_ylim(0, 0.7)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('L/min/kg', size = 12, color = 'black')
#ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MV low alarm limit', 'MV high alarm limit'], fontsize = 12, ncol  = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 2B

In [ ]:
dpi = 200
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
    
slow_measurements['DG030']['5001|RR [1/min]'].plot(ax = ax1, 
                        color = 'blue',  x_compat = True);
slow_measurements['DG030']['RR_high'].plot(ax = ax1, 
                        color = 'red', linewidth = 3, x_compat = True );
slow_measurements['DG030']['RR_set'].plot(ax = ax1, 
                        color = 'green', linewidth = 3,  x_compat = True );

ax1.set_ylim(0, 180)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('breaths/min', size = 12, color = 'black')
#ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['RR', 'RR high alarm limit', 'set RR'], fontsize = 12, ncol = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

#### Figure 2 color

In [ ]:
dpi = 200
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 9)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=0.2)

# Figure 2A

ax1 = fig.add_subplot(2, 1, 1);
    
slow_measurements['DG044']['MV_kg'].plot(ax = ax1, color = 'blue', x_compat = True )
slow_measurements['DG044']['MV_low_weight'].plot(ax = ax1, color = 'green', linewidth = 3, x_compat = True)
slow_measurements['DG044']['MV_high_weight'].plot(ax = ax1, color = 'red', linewidth = 3,  x_compat = True)

ax1.set_ylim(0, 0.7)
ax1.set_xlabel('', size = 12, color = 'black')
ax1.set_ylabel('L/min/kg', size = 12, color = 'black')
#ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MV low alarm limit', 'MV high alarm limit'], fontsize = 12, ncol  = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center');
    

# Figure 2B

ax2 = fig.add_subplot(2, 1, 2);
    
slow_measurements['DG030']['5001|RR [1/min]'].plot(ax = ax2, 
                        color = 'blue', x_compat = True);
slow_measurements['DG030']['RR_high'].plot(ax = ax2, 
                        color = 'red', linewidth = 3, x_compat = True );
slow_measurements['DG030']['RR_set'].plot(ax = ax2, 
                        color = 'green', linewidth = 3, x_compat = True );

ax2.set_ylim(0, 180)
ax2.set_xlabel('Time', size = 12, color = 'black')
ax2.set_ylabel('breaths/min', size = 12, color = 'black')
#ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.legend(['RR', 'RR high alarm limit', 'set RR'], fontsize = 12, ncol = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.05, 0.92, 'A', fontsize = 20) 
fig.text(0.05, 0.54, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2_col', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi = 200
filetype = 'tiff'

fig = plt.figure()
fig.set_size_inches(12, 9)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=0.2)

# Figure 2A

ax1 = fig.add_subplot(2, 1, 1);
    
slow_measurements['DG044']['MV_kg'].plot(ax = ax1, color = 'black', alpha = 0.6, x_compat = True )
slow_measurements['DG044']['MV_low_weight'].plot(ax = ax1, 
                                    color = 'black', alpha = 0.3, linewidth = 3, x_compat = True)
slow_measurements['DG044']['MV_high_weight'].plot(ax = ax1, 
                                    color = 'black', alpha = 0.9, linewidth = 3,  x_compat = True)

ax1.set_ylim(0, 0.7)
ax1.set_xlabel('', size = 12, color = 'black')
ax1.set_ylabel('L/min/kg', size = 12, color = 'black')
#ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MV low alarm limit', 'MV high alarm limit'], fontsize = 12, ncol  = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax1.xaxis.set_major_formatter(majorFmt)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center');
    

# Figure 2B

ax2 = fig.add_subplot(2, 1, 2);
    
slow_measurements['DG030']['5001|RR [1/min]'].plot(ax = ax2, 
                        color = 'black', alpha = 0.6, x_compat = True);
slow_measurements['DG030']['RR_high'].plot(ax = ax2, 
                        color = 'black', alpha = 0.9, linewidth = 3, x_compat = True );
slow_measurements['DG030']['RR_set'].plot(ax = ax2, 
                        color = 'black', alpha = 0.3, linewidth = 3, x_compat = True );

ax2.set_ylim(0, 180)
ax2.set_xlabel('Time', size = 12, color = 'black')
ax2.set_ylabel('breaths/min', size = 12, color = 'black')
#ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.legend(['RR', 'RR high alarm limit', 'set RR'], fontsize = 12, ncol = 3)
    
majorFmt = dates.DateFormatter('%H:%M')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.05, 0.92, 'A', fontsize = 20) 
fig.text(0.05, 0.54, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2_bw', filetype), dpi=dpi, format= filetype, 
            bbox_inches='tight', pad_inches=0.1);